In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [ ]:
df = pd.read_csv('/kaggle/input/heart-disease/heart.csv')

# Exploratory Data Analysis

We can see using .head() that all features are numeric

In [ ]:
df.head()

Using .info() we see that none of the columns contain NaN values

In [ ]:
df.info()

We can create a heatmap comparing the correlation bewteen all pairwise features since they are all numeric

In [ ]:
sns.heatmap(df.corr())

# Feature Engineering and Model Building

Before we apply scaling to some of the numeric features, we create a RandomForest model using the bagging technique. We split the data we have into 80% train and 20% test, and further split train into 80% train and 20% validation.

In [ ]:
X = df.loc[:, df.columns != 'target']
y = df.loc[:, 'target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

model1 = RandomForestRegressor()
bags = 10
seed = 1
bagged_prediction = np.zeros(X_val.shape[0])
for n in range(0, bags):
    model1.set_params(random_state = seed + n) 
    model1.fit(X_train, y_train)
    preds1 = model1.predict(X_val)
    bagged_prediction += preds1
bagged_prediction /= bags
test_preds1 = model1.predict(X_test)

Now we take replace some features that showed skewness in the pairplot above with its natural log

In [ ]:
df['log_cp'] = np.log(df['cp'] + 1)
df['log_ca'] = np.log(df['ca'] + 1)
df['log_oldpeak'] = np.log(df['oldpeak'] + 1)
df['log_trestbps'] = np.log(df['trestbps'] + 1)
df['log_thalach'] = np.log(df['thalach'] + 1)
df.drop('cp', axis=1, inplace=True)
df.drop('ca', axis=1, inplace=True)
df.drop('oldpeak', axis=1, inplace=True)
df.drop('trestbps', axis=1, inplace=True)
df.drop('thalach', axis=1, inplace=True)

Our second model uses Linear Regression with the scaled values. Then we stack our predictions from the two models and take their average to create a meta model which we will use to generate our final predictions

In [ ]:
X = df.loc[:, df.columns != 'target']
y = df.loc[:, 'target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

model2 = LinearRegression().fit(X_train, y_train)
preds2 = model2.predict(X_val)
test_preds2 = model2.predict(X_test)

stacked_predictions = np.column_stack((preds1, preds2))
stacked_test_predictions = np.column_stack((test_preds1, test_preds2))
meta_model = LinearRegression()
meta_model.fit(stacked_predictions, y_val)
final_predictions = meta_model.predict(stacked_test_predictions)

Since the final predictions are averaged, it will fall on a continuous scale. The target variables are either 0 or 1 so we apply a threshold of 0.5 to determine its value

In [ ]:
final_predictions = (final_predictions >= 0.5).astype(int)
final_predictions

In [ ]:
f1_score(final_predictions, y_test)